## TODO: rename paths and files!

## download historical events from opendata
https://data.cityofnewyork.us/City-Government/NYC-Permitted-Event-Information-Historical/bkfu-528j

In [1]:
import pandas as pd
import polars as pl
import json

In [2]:
def augment_weather(df: pl.DataFrame, year, parquet = False):
    tmp = df
    
    # transpose if reading parquet?
    if parquet:
        tmp = pd.DataFrame(df).transpose()
        tmp.columns = df.columns

    weather = pd.read_csv("./data/weather/ny_weather_{0}.csv".format(year))
    weather = weather[['datetime', 'temp', 'humidity', 'snowdepth', 'windspeed', 'conditions', 'description']]
    weather['snowdepth'].fillna(0, inplace=True)
    
    weather['datetime'] = pd.to_datetime(weather['datetime']).dt.date
    
    tmp['datetime'] = tmp['datetime_issue'].dt.date

    res = tmp.merge(weather, on='datetime', how='left')

    out = pd.DataFrame(res)
    return out

In [3]:
df = pd.read_csv("data/2023_parking_violations.csv")

C:\Users\Ana\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (17,18,20,22,23,26,27,29,30,31,32,36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df['datetime_issue']= df['Issue Date'] + ' ' + df['Violation Time']
df['datetime_issue'] = df['datetime_issue'] + 'M'
df['datetime_issue'] = pd.to_datetime(df['datetime_issue'], format='%m/%d/%Y %I%M%p', errors='coerce')



In [5]:
df['datetime_issue'].min()

Timestamp('1972-04-04 09:45:00')

In [6]:
df = augment_weather(df, 2023)

In [7]:
df.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Hydrant Violation,Double Parking Violation,datetime_issue,datetime,temp,humidity,snowdepth,windspeed,conditions,description
0,1484697303,JER1863,NY,PAS,06/10/2022,67,SDN,TOYOT,P,34330,...,NaN,NaN,2022-06-10 10:37:00,2022-06-10,22.3,49.5,0.0,21.7,Rain,Clear conditions throughout the day with late ...
1,1484697315,KEV4487,NY,PAS,06/13/2022,51,SUBN,JEEP,K,34310,...,NaN,NaN,2022-06-13 10:45:00,2022-06-13,24.3,72.4,0.0,14.5,"Rain, Partially cloudy",Becoming cloudy in the afternoon with rain.
2,1484697625,H73NYD,NJ,PAS,06/19/2022,63,SDN,JEEP,N,30640,...,NaN,NaN,2022-06-19 11:16:00,2022-06-19,17.6,40.6,0.0,29.5,Partially cloudy,Partly cloudy throughout the day.
3,1484697674,GJC9296,NY,PAS,06/19/2022,63,SUBN,LEXUS,N,30640,...,NaN,NaN,2022-06-19 10:52:00,2022-06-19,17.6,40.6,0.0,29.5,Partially cloudy,Partly cloudy throughout the day.
4,1484697686,M51PUV,NJ,PAS,06/19/2022,63,SDN,HYUND,N,30640,...,NaN,NaN,2022-06-19 11:07:00,2022-06-19,17.6,40.6,0.0,29.5,Partially cloudy,Partly cloudy throughout the day.


In [8]:
df['Street Name'] = df['Street Name'].astype(str)

In [9]:
f = open('data/street-suffix-abbreviations.json')
street_suffix = json.load(f)

In [10]:
def fix_street_abbreviation(x):
    abb = x.split()[-1]

    if abb in street_suffix:
        x = x.replace(abb, street_suffix[abb][0])
    return x

In [11]:
df["Street Name"] = df.apply(lambda row: fix_street_abbreviation(row['Street Name']), axis = 1)

In [12]:
df["Street Name"][:5]

0    W 28TH STREET
1       27TH DRIVE
2     SOUTH STREET
3     SOUTH STREET
4     SOUTH STREET
Name: Street Name, dtype: object

In [13]:
schools = pd.read_csv('./data/school_locations.csv')

In [14]:
def street_from_address(x):
    # make sure the addresses are written correctly
    a = x.split(" ")[1:]
    return ' '.join(a)

In [15]:
schools['Street Name'] = schools.apply(lambda row: street_from_address(row['primary_address_line_1']), axis = 1)

In [16]:
# extract relevant categories
schools = schools[["Street Name", "Location_Category_Description", "Police_precinct"]]

# note we are choosing to rename to issuer precinct. 
# we could also join via Violation Precinct - it would give us a different picture
schools = schools.rename(columns={"Police_precinct": "Issuer Precinct"}, errors="raise")

In [17]:
schools_g = schools.groupby(['Street Name', 'Issuer Precinct']).count()
schools_g = schools_g.reset_index()

In [18]:
df = pd.merge(df, schools_g, how = "left", on=["Street Name", "Issuer Precinct"])

In [19]:
len(df)

14392670

In [20]:
business = pd.read_csv('./data/legally_operating_businesses.csv')

C:\Users\Ana\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
# filter data - active businesses only, ONLY in new york
business = business[(business["License Type"] == "Business") & (business["License Status"] == "Active") & (business["Address State"] == "NY") & (business["Address Borough"] != "Outside NYC")]

In [22]:
business["Address Borough"] = business["Address Borough"].str.upper()

In [23]:
business["Address Street Name"] = business["Address Street Name"].str.upper()
business["Address Street Name"] = business["Address Street Name"].astype('str')

In [24]:
business["Address Street Name"] = business.apply(lambda row: fix_street_abbreviation(row['Address Street Name']), axis = 1)

In [25]:
business = business.rename(columns={"Address Street Name": "Street Name"}, errors="raise")

In [26]:
business['Business'] = 1

In [27]:
def county_to_borough(x):
    if (x == 'K') | (x.upper() == 'KINGS') | (x == 'BK'):
        return 'BROOKLYN'
    elif (x == 'BX') | (x.upper() == 'BRONX'):
        return 'BRONX'
    elif (x == 'R') | (x.upper() == 'RICH') | (x == 'ST') | (x.upper() == 'RICHM'):
        return 'STATEN ISLAND'
    elif (x == 'NY') | (x == 'MN'):
        return 'MANHATTAN'
    elif (x == 'Q') | (x == 'QN') | (x.upper() == 'QNS'):
        return 'QUEENS'
    else:
        return x    

In [28]:
df["Violation County"] = df["Violation County"].astype('str')
df["Address Borough"] = df.apply(lambda row: county_to_borough(row['Violation County']), axis = 1)

In [29]:
business = business[["Street Name", "Address Borough", "Business"]]

In [30]:
business_g = business.groupby(['Street Name', 'Address Borough']).count()
business_g = business_g.reset_index()

In [31]:
df = pd.merge(df, business_g, how = "left", on=["Street Name", "Address Borough"])

In [32]:
df['Business'] = df['Business'].fillna(0)

In [33]:
header = pd.read_csv("data/nyc_permitted_events.csv", nrows=1)

In [34]:
# read only last 1 million events for 2022, 2023
events = pd.read_csv("data/nyc_permitted_events.csv", header=0, skiprows=16000000)

In [35]:
events.columns = header.columns

In [36]:
events.head()

,Event ID,Event Name,Start Date/Time,End Date/Time,Event Agency,Event Type,Event Borough,Event Location,Event Street Side,Street Closure Type,Community Board,Police Precinct
0,640309,Soccer -Regulation,10/16/2022 04:00:00 PM,10/16/2022 08:00:00 PM,Parks Department,Sport - Youth,Queens,Forest Park: Soccer-01,NaN,NaN,"82,","102,"
1,664352,Soccer -Regulation,10/02/2022 10:00:00 AM,10/02/2022 04:00:00 PM,Parks Department,Sport - Adult,Bronx,Ferry Point Park: Soccer-01,NaN,NaN,"10,","45,"
2,655548,Baseball - 12 and Under (Little League),09/27/2022 09:00:00 AM,09/27/2022 10:00:00 PM,Parks Department,Sport - Youth,Bronx,"Co-op City Field: Softball-01 ,Co-op City Fiel...",NaN,NaN,"10,","45,"
3,639967,Kickball,09/29/2022 12:00:00 PM,09/29/2022 07:00:00 PM,Parks Department,Sport - Adult,Manhattan,Tompkins Square Park: Softball-01,NaN,NaN,"03,","9,"
4,645334,Soccer - Non Regulation,09/26/2022 05:00:00 PM,09/26/2022 07:00:00 PM,Parks Department,Sport - Youth,Queens,"Juniper Valley Park: Softball-03 ,Juniper Vall...",NaN,NaN,"05,","104,"


In [37]:
events['Start Date/Time'][:5]

0    10/16/2022 04:00:00 PM
1    10/02/2022 10:00:00 AM
2    09/27/2022 09:00:00 AM
3    09/29/2022 12:00:00 PM
4    09/26/2022 05:00:00 PM
Name: Start Date/Time, dtype: object

In [38]:
events['parsed police'] = events.apply(lambda x: str(x['Police Precinct']).split(",")[0], axis=1, result_type='expand')

In [39]:
events['Start Date/Time'] = pd.to_datetime(events['Start Date/Time'], format='%m/%d/%Y %I:%M:%S %p',)


In [40]:
events['Start Date/Time']

0        2022-10-16 16:00:00
1        2022-10-02 10:00:00
2        2022-09-27 09:00:00
3        2022-09-29 12:00:00
4        2022-09-26 17:00:00
                 ...        
936385   2023-05-01 18:00:00
936386   2023-04-22 08:00:00
936387   2023-04-25 15:30:00
936388   2023-05-18 08:00:00
936389   2023-04-29 08:00:00
Name: Start Date/Time, Length: 936390, dtype: datetime64[ns]

In [41]:
events['End Date/Time'] = pd.to_datetime(events['End Date/Time'], format='%m/%d/%Y %I:%M:%S %p',)

In [42]:
events['Start Date'] = events['Start Date/Time'].dt.date

In [43]:
events['End Date'] = events['End Date/Time'].dt.date

In [44]:
events = events.rename(columns={"parsed police": "Issuer Precinct"}, errors="raise")

In [46]:
events['event'] = 1

In [58]:
events = events[['Issuer Precinct', 'event', 'Start Date']]

In [59]:
df = df.rename(columns={"datetime": "Start Date"}, errors="raise")

In [60]:
events

,Issuer Precinct,event,Start Date
0,102,1,2022-10-16
1,45,1,2022-10-02
2,45,1,2022-09-27
3,9,1,2022-09-29
4,104,1,2022-09-26
...,...,...,...
936385,13,1,2023-05-01
936386,103,1,2023-04-22
936387,78,1,2023-04-25
936388,122,1,2023-05-18


In [61]:
events = events[events['Issuer Precinct'] != 'nan']

In [62]:
#events = events.dropna(subset=['Issuer Precinct'])
events['Issuer Precinct'] = events['Issuer Precinct'].astype(int)

In [63]:
events_g = events.groupby(['Issuer Precinct', 'Start Date']).count()
events_g = events_g.reset_index()

In [64]:
len(events_g)

14541

In [65]:
events_g.head()

,Issuer Precinct,Start Date,event
0,1,2022-05-20,6
1,1,2022-09-22,2
2,1,2022-09-23,4
3,1,2022-09-24,3
4,1,2022-09-25,10


In [66]:
df = pd.merge(df, events_g, how = "left", on=["Issuer Precinct", "Start Date"])

In [67]:
df.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,temp,humidity,snowdepth,windspeed,conditions,description,Location_Category_Description,Address Borough,Business,event
0,1484697303,JER1863,NY,PAS,06/10/2022,67,SDN,TOYOT,P,34330,...,22.3,49.5,0.0,21.7,Rain,Clear conditions throughout the day with late ...,NaN,MANHATTAN,11.0,NaN
1,1484697315,KEV4487,NY,PAS,06/13/2022,51,SUBN,JEEP,K,34310,...,24.3,72.4,0.0,14.5,"Rain, Partially cloudy",Becoming cloudy in the afternoon with rain.,NaN,MANHATTAN,0.0,NaN
2,1484697625,H73NYD,NJ,PAS,06/19/2022,63,SDN,JEEP,N,30640,...,17.6,40.6,0.0,29.5,Partially cloudy,Partly cloudy throughout the day.,NaN,MANHATTAN,4.0,NaN
3,1484697674,GJC9296,NY,PAS,06/19/2022,63,SUBN,LEXUS,N,30640,...,17.6,40.6,0.0,29.5,Partially cloudy,Partly cloudy throughout the day.,NaN,MANHATTAN,4.0,NaN
4,1484697686,M51PUV,NJ,PAS,06/19/2022,63,SDN,HYUND,N,30640,...,17.6,40.6,0.0,29.5,Partially cloudy,Partly cloudy throughout the day.,NaN,MANHATTAN,4.0,NaN


In [68]:
df.to_parquet('parking_augmented.parquet')

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.